# 1. Optimization problem

## 1.1. Problem statement
Optimization is the problem of finding minimum or maximum of a function.

## 1.2. First-derivative test

## 1.3. Second-derivative test

# 2. Gradient Descent
Function to minimize: $f({\theta})$

## 2.1. The basic algorithm

$$\mathbf{x}\leftarrow\mathbf{x}-\eta\nabla f(\mathbf{x})$$

$$\mathbf{x}^{(t+1)}=\mathbf{x}^{(t)}-\eta\nabla f(\mathbf{x}^{(t)})$$

$$\mathbf{x}_{t+1}=\mathbf{x}_t-\eta\nabla f(\mathbf{x}_t)$$

## 2.2. Batch and epoch

## 2.3. Evolutionary map

## 2.4. Momentum

## 2.5. Nesterov accelerated

## 2.6. AdaGrad

## 2.7. AdaDelta

## 2.8. RMSprop

## 2.9. Adam

## 2.10. Nadam

## 2.11. AdaMax

## 2.12. AMSGrad

# 3. Bayesian Optimization

## 3.1. Gaussian Process

## 3.2. Bayesian Optimization

## 3.3. Hyperopt implementation

## 3.4. Optuna implementation